In [ ]:
import json
import numpy
import os
import pandas
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [ ]:
# CHANGE ME
COMPANY_IDENTIFIER = 'CI'
TRANSFER_PACKAGES_START_DATE = '2021-01-01'

## Queries 1

In [ ]:
company_licenses_query = create_queries.create_company_licenses_query(COMPANY_IDENTIFIER)
company_download_summaries_query = create_queries.create_company_download_summaries_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)

company_licenses_dataframe = pandas.read_sql_query(company_licenses_query, engine)
company_download_summaries_dataframe = pandas.read_sql_query(company_download_summaries_query, engine)

## Licenses

In [ ]:
company_licenses_dataframe

## Download summaries

In [ ]:
license_numbers = company_download_summaries_dataframe['license_number'].unique()
download_summary_records = company_download_summaries_dataframe.to_dict('records')

license_number_to_download_summary_records = {}

for license_number in license_numbers:
    license_number_to_download_summary_records[license_number] = list(filter(
        lambda download_summary_record: download_summary_record['license_number'] == license_number,
        download_summary_records
    ))

bad_count = 0

for license_number, download_summary_records in license_number_to_download_summary_records.items():
    print(f'Verifying download summaries for license {license_number}...')
    print(f'Earliest download summary: {download_summary_records[-1]["date"]}')
    print(f'Latest download summary: {download_summary_records[0]["date"]}')
    for download_summary_record in download_summary_records:
        if download_summary_record['status'] != 'completed':
            bad_count += 1
            print(f'Found bad download summary for license {license_number} on date {download_summary_record["date"]}')
    print('')

if bad_count > 0:
    print(f'[FAILURE] Found a total of {bad_count} bad download summaries')
else:
    print(f'[SUCCESS] All download summaries look good!')

## Queries 2

In [ ]:
company_license_records = company_licenses_dataframe.to_dict('records')
# List of manufacturer license numbers.
license_numbers = []
for company_license_record in company_license_records:
    license_number = company_license_record['license_number']
    license_category = company_license_record['license_category']
    if license_category == 'Distributor':
        license_numbers.append(license_number)
license_numbers

In [ ]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=license_numbers,
)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=license_numbers,
)
company_unknown_transfer_packages_query = create_queries.create_company_unknown_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
#     license_numbers=license_numbers,
)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
    license_numbers=license_numbers,
)

company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_unknown_transfer_packages_dataframe = pandas.read_sql_query(company_unknown_transfer_packages_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)

## Transfer packages

In [ ]:
company_unknown_transfer_package_records = company_unknown_transfer_packages_dataframe.to_dict('records')
unknown_count = len(company_unknown_transfer_package_records)
if unknown_count > 0:
    print(f'[FAILURE] Found a total of {unknown_count} unknown transfer packages')
else:
    print(f'[SUCCESS] No unknown transfer packages!')

In [ ]:
fn = lambda row: f'{row.created_date.year}-{"0" if row.created_date.month < 10 else ""}{row.created_date.month}'
col = company_incoming_transfer_packages_dataframe.apply(fn, axis=1)
company_incoming_transfer_packages_dataframe = company_incoming_transfer_packages_dataframe.assign(created_month=col.values)

In [ ]:
# Incoming transfer packages by shipment type name
company_incoming_transfer_packages_dataframe.groupby(['created_month', 'shipment_type_name'])['package_id'].count().unstack().plot.bar(figsize=(24, 8), stacked=True)

In [ ]:
# Incoming transfer packages by standard vs wholesale
company_incoming_transfer_packages_dataframe.groupby(['created_month', 'shipment_transaction_type'])['package_id'].count().unstack().plot.bar(figsize=(24, 8), stacked=True)

In [ ]:
# % incoming transfer packages with receiver wholesale price

rwp_exists_count = len(company_incoming_transfer_packages_dataframe[company_incoming_transfer_packages_dataframe['receiver_wholesale_price'].notnull()].index)
total_count = len(company_incoming_transfer_packages_dataframe.index)

print(f'{round(rwp_exists_count / total_count * 100, 2)}% of incoming transfer packages have receiver wholesale price')

## Compare computed inventory vs Metrc inventory (metrc_packages)

In [ ]:
TODAY_DATE = date.today().strftime('%m/%d/%Y')
INVENTORY_DATES = [TODAY_DATE]
ANALYSIS_PARAMS = {
    'sold_threshold': 1.0,
    'find_parent_child_relationships': False,
}
print('Today is {}'.format(TODAY_DATE))

In [ ]:
analysis_ctx = inventory_types.AnalysisContext(
    output_root_dir='tmp',
    read_params=inventory_types.ReadParams(
        use_cached_dataframes=False
    ),
    write_params=inventory_types.WriteOutputParams(
        save_download_dataframes=False
    )
)

d = util.Download()
sql_helper = util.BigQuerySQLHelper(
    ctx=analysis_ctx,
    engine=engine,
)
d.download_dataframes(
    all_dataframes_dict=download_util.AllDataframesDict(
        incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
        outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
        sales_transactions_dataframe=company_sales_transactions_dataframe,
        sales_receipts_dataframe=company_sales_receipts_dataframe,
        inventory_packages_dataframe=company_inventory_packages_dataframe,
    ),
    sql_helper=sql_helper
)

q = util.Query(
    inventory_dates=[], # gets filled in once we have the dataframes
    transfer_packages_start_date=TRANSFER_PACKAGES_START_DATE,
    sales_transactions_start_date=SALES_TRANSACTIONS_START_DATE,
    company_name=COMPANY_IDENTIFIER,
    company_identifier=COMPANY_IDENTIFIER,
    license_numbers=[],
)

id_to_history = util.get_histories(d, ANALYSIS_PARAMS)
util.print_counts(analysis_ctx, id_to_history)

In [ ]:
date_to_inventory_packages_dataframe = {}

id_to_history = util.get_histories(d, ANALYSIS_PARAMS)
inventory_valuations = []

for inventory_date in INVENTORY_DATES:
    computed_inventory_package_records = util.create_inventory_dataframe_by_date(
        id_to_history, inventory_date, params=ANALYSIS_PARAMS)    
    computed_inventory_packages_dataframe = pandas.DataFrame(
        computed_inventory_package_records,
        columns=util.get_inventory_column_names(),
    )
    date_to_inventory_packages_dataframe[inventory_date] = computed_inventory_packages_dataframe
    inventory_valuations.append(valuations_util.get_total_valuation_for_date(
        computed_inventory_packages_dataframe=computed_inventory_packages_dataframe,
        company_incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
        inventory_date=inventory_date,
        using_nb=True,
    ))

In [ ]:
from_packages_inventory_dataframe = company_inventory_packages_dataframe[[
    'package_id',
    'packaged_date',
    'unit_of_measure',
    'product_category_name',
    'product_name',
    'quantity',
]].sort_values('package_id')

package_id_to_actual_row = {}
for index, row in from_packages_inventory_dataframe.iterrows():
    package_id_to_actual_row[str(row['package_id'])] = row
    
res = util.compare_inventory_dataframes(
    ctx=analysis_ctx,
    computed=date_to_inventory_packages_dataframe[TODAY_DATE],
    actual=from_packages_inventory_dataframe,
    options={
        'num_errors_to_show': 10,
        'accept_computed_when_sold_out': True
    }
)

inventory_cost_valuation = valuations_util.get_inventory_valuation(
    inventory_packages_dataframe=company_inventory_packages_dataframe,
    incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
)
print(f'Cost valuation of Metrc-reported inventory as of today: ${round(inventory_cost_valuation, 2)}')

## Export data

In [ ]:
# Flip flag if you want to export data to files.
is_export_enabled = True

In [ ]:
import time
from datetime import date

TODAY_DATE = date.today().strftime('%m-%d-%Y')
NOW = int(time.time())

export_incoming_transfer_packages_dataframe = company_incoming_transfer_packages_dataframe

date_columns = export_incoming_transfer_packages_dataframe.select_dtypes(include=['datetime64[ns, UTC]']).columns
for date_column in date_columns:
    export_incoming_transfer_packages_dataframe[date_column] = export_incoming_transfer_packages_dataframe[date_column].dt.date

incoming_transfer_packages_file_name = f'~/Downloads/{COMPANY_IDENTIFIER}_incoming_transfer_packages_{TODAY_DATE}_{NOW}'

export_inventory_packages_dataframe = company_inventory_packages_dataframe

date_columns = export_inventory_packages_dataframe.select_dtypes(include=['datetime64[ns, UTC]']).columns
for date_column in date_columns:
    export_inventory_packages_dataframe[date_column] = export_inventory_packages_dataframe[date_column].dt.date

inventory_packages_file_name = f'~/Downloads/{COMPANY_IDENTIFIER}_inventory_packages_{TODAY_DATE}_{NOW}'

if is_export_enabled:
    num_incoming_transfer_packages = len(export_incoming_transfer_packages_dataframe.index)
    print(f'Exporting {num_incoming_transfer_packages} incoming transfer packages to files...')
    export_incoming_transfer_packages_dataframe.reset_index().to_excel(f'{incoming_transfer_packages_file_name}.xlsx')
    export_incoming_transfer_packages_dataframe.reset_index().to_csv(f'{incoming_transfer_packages_file_name}.csv')
    print(f'Exported {num_incoming_transfer_packages} incoming transfer packages to files')

    num_inventory_packages = len(export_inventory_packages_dataframe.index)
    print(f'Exporting {num_inventory_packages} inventory packages to files...')
    export_inventory_packages_dataframe.reset_index().to_excel(f'{inventory_packages_file_name}.xlsx')
    export_inventory_packages_dataframe.reset_index().to_csv(f'{inventory_packages_file_name}.csv')
    print(f'Exported {num_inventory_packages} inventory packages to files')